# Generate Tests

This notebook imports a compiled ThermoCodegen object and uses the `Tester` class to generate pandas dataframes
and pytest files for testing all of the interfaces to the code generated objects

## testing notebook for the Fo-SiO$_2$ system

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from glob import glob
import pytest
import warnings

from thermocodegen.testing import Tester

Set a time-stamp for the tests


In [ ]:
import thermocodegen as tcg
timestamp = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
TCG_VERSION = tcg.__version__

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../..')
TEST_DIR = HOME_DIR+'/tests/test-v{}-{}'.format(TCG_VERSION,timestamp)
try:
    os.makedirs(TEST_DIR)
except:
    pass
print(TEST_DIR)

In [ ]:
# load reaction object
import py_fo_sio2_poly_linear_rxns as pfb
rxn = pfb.fo_sio2_poly_linear_rxns()

In [ ]:
rxn.report()

In [ ]:
tester = Tester.factory(pfb)

In [ ]:
tester.phase_dict

## Test endmembers

### set input dataframe for endmembers

In [ ]:
em_df_in = tester.get_endmember_df_in()
em_df_in

### just set P,T fields for testing

In [ ]:
# Set test variables
T=1700.
P=1000.
em_df_in['P']= P*np.ones(len(em_df_in))
em_df_in['T']= T*np.ones(len(em_df_in))
em_df_in.dropna(axis=1,inplace=True)
em_df_in

### Generate output testing dataframe and pytest files

In [ ]:
em_df_out = tester.set_df_out(em_df_in)
em_df_out

In [ ]:
em_df_out.to_csv('{}/Endmembers.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_endmembers'.format(TEST_DIR) 
tester.write_tests(em_df_out,fileroot=fileroot)

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Phases

### set input dataframe for phases

In [ ]:
ph_df_in = tester.get_phase_df_in()
ph_df_in

In [ ]:
ph_df_in.drop([0,4],inplace=True)
ph_df_in

### just set P,T,c,x,n fields for testing

In [ ]:
rxn.report()

In [ ]:
# Set test variables
T=1700.
P=1000.
ilq = 0
iol = 1
iopx = 2
isi = 3
i_c = 0

phases = rxn.phases()
pdict = {}
for i,phase in enumerate(phases):
    pdict[phase.name()] = i
pdict

In [ ]:
C=rxn.zero_C()
C[ilq] = [.25, .75]
for i in range(1,4):
    C[i] = [1.]
C

In [ ]:
X = rxn.C_to_X(C)
E_dict = {}
for i,phase in enumerate(phases):
    E_dict[phase.name()] = np.array(phase.conv_moles_to_elm(X[i]))

In [ ]:
ph_df_in['P']= P*np.ones(len(C))
ph_df_in['T']= T*np.ones(len(C))
for index,row in ph_df_in.iterrows():
    name = row['name']
    print(index,name,pdict[name])
    ph_df_in['c'][index] = np.array(C[pdict[name]])
    ph_df_in['x'][index] = np.array(X[pdict[name]])
    ph_df_in['n'][index] = np.array(X[pdict[name]])
    ph_df_in['e'][index] = E_dict[name]
ph_df_in['i']=i_c
ph_df_in.dropna(axis=1,inplace=True)
ph_df_in

### Generate output testing dataframe and pytest files

In [ ]:
ph_df_out = tester.set_df_out(ph_df_in)
ph_df_out

In [ ]:
ph_df_out.to_csv('{}/Phases.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_phases'.format(TEST_DIR) 
tester.write_tests(ph_df_out,fileroot=fileroot)

In [ ]:
file=glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Reactions

### set input dataframe for reactions

In [ ]:
rx_df_in = tester.get_reaction_df_in()
rx_df_in

### just set P,T,c,x,n fields for testing

In [ ]:
rxn.report()

In [ ]:
# Set test variables
X = rxn.C_to_X(C)
Phi = [0.1, 0.6, 0.3, 0.0 ]

# Variables for parameter set and get functions
p, val = "R", 8.314

#jacobian terms
j = 0
i = 3
l = 0
k = 0
m = 1

In [ ]:
rx_df_in['P']= P
rx_df_in['T']= T
rx_df_in['C']= [C]
rx_df_in['X']= [X]
rx_df_in['Phi'] = [Phi]
rx_df_in['p'] = p
rx_df_in['val'] = val
rx_df_in[['i','j','k','l','m']]=pd.DataFrame.from_dict(dict(i=[i],j=[j],k=[k],l=[l],m=[m]))
rx_df_in


### Generate output testing dataframe and pytest files

In [ ]:
rx_df_out = tester.set_df_out(rx_df_in)
rx_df_out

In [ ]:
rx_df_out.to_csv('{}/Reactions.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_rxns'.format(TEST_DIR)
tester.write_tests(rx_df_out,fileroot=fileroot)

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])